In [2]:
import math

In [ ]:
@cuda.jit(device=False)
def scan_preScan(array):
    threadID = cuda.threadIdx.x
    blockID = cuda.blockIdx.x # Block id in a 1D grid

    idx_shared = threadID*2 # index in shared array 
    idx_device = idx_shared + blockID*scan_array_size # index in full array
    nrows = array.shape[0] - 1 # index limit for last block
    
    # Preload data into shared memory
    # s_array = cuda.shared.array(shape=(scan_array_size), dtype=array.dtype) 
    s_array = cuda.shared.array(shape=(scan_array_size), dtype=np.int32) 
    if (idx_device < nrows): # each thread handles 2 fields
        s_array[idx_shared] = array[idx_device]
        s_array[idx_shared + 1] = array[idx_device + 1] 
    cuda.syncthreads()  # Wait until all threads finish preloading

    exclusiv_parallel_prefix_sum(s_array, idx_shared, scan_array_size) # scan s_array
    cuda.syncthreads()
    
    # output for exclusiv scan
    # if (idx2 < nrows):
    #   array[idx2] = s_array[idx]
    #   array[idx2 + 1] = s_array[idx + 1] 
    # output for inclusiv scan: switch the exclisiv scan to an inclusiv scan by shifting left and adding at the end
    if (idx_shared + 2 == scan_array_size or idx_device + 1 == nrows): # last index in s_array or last index in array
        array[idx_device] = s_array[idx_shared + 1]
        array[idx_device + 1] += s_array[idx_shared + 1]
    elif (idx_device < nrows):
        array[idx_device] = s_array[idx_shared + 1]
        array[idx_device + 1] = s_array[idx_shared + 2] 

In [3]:
scan_max_threads = 1024 # number of threads per block the card can handle (select max possible value)
scan_array_size = scan_max_threads*2 # array size which can be accumulated parallel by one block
scan_nlevel = int(math.log2(scan_array_size))

In [50]:
array_shape = 226182

threadID = 450
blockID = 110

idx_shared = threadID*2
idx_device = idx_shared + blockID*scan_array_size
nrows = array_shape - 1

bIsTrue = idx_device < nrows

print(idx_shared)
print(idx_shared + 1)


900
901


In [51]:
print(idx_device)
print(idx_device + 1)

226180
226181


In [54]:
idx_shared + 2 == scan_array_size

False

In [55]:
idx_device + 1 == nrows

True

In [56]:
idx_device

226180

In [58]:
idx_shared + 1

901

In [63]:
idx_shared = (scan_array_size - 2)
idx_device = idx_shared + blockID*scan_array_size
print(idx_device)

227326


In [65]:
227326 - array_shape

1144

In [64]:
array_shape

226182

In [ ]:
idx_shared + 2 == scan_array_size